In [5]:
from openai import OpenAI
api_key = "sk-None-kOYR3Bk2Fs8xnh5UhMBvT3BlbkFJGhjP7VUfURDIhEziNpED"

**Data Ingestion**

In [12]:
import requests
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

In [30]:
import wikipediaapi

def fetch_wikipedia_article(article_title):
    # Create a Wikipedia API object with a specified user agent
    user_agent = "MyApp/1.0 (myemail@example.com)"
    wiki_wiki = wikipediaapi.Wikipedia('en', headers={'User-Agent': user_agent})

    # Fetch the Wikipedia page
    page = wiki_wiki.page(article_title)

    # Check if the page exists
    if page.exists():
        # Return the raw text content of the page
        return page.text
    else:
        return f"The article '{article_title}' does not exist on Wikipedia."

In [127]:
article_title = "Cryptocurrency"
article_text = fetch_wikipedia_article(article_title)

with open("Cryptocurrency.txt", "w", encoding="utf-8") as f:
    f.write(article_text)

In [129]:
# Read the article text from the file
with open("Cryptocurrency.txt", "r", encoding="utf-8") as f:
    document = f.read()

In [131]:
print(document[:1000])

A cryptocurrency, crypto-currency, or crypto is a digital currency designed to work as a medium of exchange through a computer network that is not reliant on any central authority, such as a government or bank, to uphold or maintain it. 
Individual coin ownership records are stored in a digital ledger, which is a computerized database using strong cryptography to secure transaction records, control the creation of additional coins, and verify the transfer of coin ownership. Despite the term that has come to describe many of the fungible blockchain tokens that have been created, cryptocurrencies are not considered to be currencies in the traditional sense, and varying legal treatments have been applied to them in various jurisdicitons, including classification as commodities, securities, and currencies. Cryptocurrencies are generally viewed as a distinct asset class in practice. Some crypto schemes use validators to maintain the cryptocurrency. In a proof-of-stake model, owners put up t

**Text Split using LangChain**

In [134]:
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

document = Document(page_content=document)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_chunks = text_splitter.split_documents([document])

Created a chunk of size 2252, which is longer than the specified 1000
Created a chunk of size 1592, which is longer than the specified 1000
Created a chunk of size 2037, which is longer than the specified 1000
Created a chunk of size 1422, which is longer than the specified 1000
Created a chunk of size 1039, which is longer than the specified 1000
Created a chunk of size 2662, which is longer than the specified 1000
Created a chunk of size 1942, which is longer than the specified 1000
Created a chunk of size 1529, which is longer than the specified 1000
Created a chunk of size 1006, which is longer than the specified 1000
Created a chunk of size 1498, which is longer than the specified 1000
Created a chunk of size 3042, which is longer than the specified 1000
Created a chunk of size 1497, which is longer than the specified 1000
Created a chunk of size 1233, which is longer than the specified 1000
Created a chunk of size 3884, which is longer than the specified 1000
Created a chunk of s

In [136]:
for i, chunk in enumerate(text_chunks):
    print(f"Chunk {i+1}:")
    print(chunk.page_content)
    print("\n" + "-"*80 + "\n")

Chunk 1:
A cryptocurrency, crypto-currency, or crypto is a digital currency designed to work as a medium of exchange through a computer network that is not reliant on any central authority, such as a government or bank, to uphold or maintain it. 
Individual coin ownership records are stored in a digital ledger, which is a computerized database using strong cryptography to secure transaction records, control the creation of additional coins, and verify the transfer of coin ownership. Despite the term that has come to describe many of the fungible blockchain tokens that have been created, cryptocurrencies are not considered to be currencies in the traditional sense, and varying legal treatments have been applied to them in various jurisdicitons, including classification as commodities, securities, and currencies. Cryptocurrencies are generally viewed as a distinct asset class in practice. Some crypto schemes use validators to maintain the cryptocurrency. In a proof-of-stake model, owners

In [143]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [145]:
embeddings=OpenAIEmbeddings(openai_api_key=api_key)

In [146]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [149]:
retriever=vectorstore.as_retriever()

In [151]:
from langchain.prompts import ChatPromptTemplate

In [153]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [155]:
prompt=ChatPromptTemplate.from_template(template)

In [157]:
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [159]:
output_parser=StrOutputParser()

In [161]:
llm_model=ChatOpenAI(openai_api_key=api_key,model_name="gpt-3.5-turbo")

In [162]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [167]:
rag_chain.invoke("What are the popular cryptocurrency")

'The popular cryptocurrencies include Bitcoin, Litecoin, Ethereum, Cardano, and Ripple (XRP). Altcoins like Litecoin and Ethereum have unique features such as faster transaction processing and smart contract functionality. Cryptocurrency prices are highly volatile, with significant market price rallies often referred to as "altseason." Cryptocurrencies are primarily used outside traditional banking and governmental institutions. The cryptocurrency sector has experienced periods of growth, bubbles, and market crashes. Countries like El Salvador and Cuba have taken steps to recognize and regulate cryptocurrencies. Ethereum underwent a consensus mechanism upgrade known as "the Merge" to reduce energy consumption. FTX Trading Ltd., a cryptocurrency exchange, filed for bankruptcy in November 2022, prompting calls for regulatory intervention to protect investors.'

In [169]:
rag_chain.invoke("What is the optimal way to invest in cryptocurrency")

'The optimal way to invest in cryptocurrency is to conduct thorough research on the market and specific cryptocurrencies. Cryptocurrencies are primarily exchanged over the Internet and are not regulated by banking or governmental institutions. Market capitalization is calculated by multiplying the price by the number of coins in circulation. Bitcoin historically dominates the market cap, with altcoins fluctuating based on Bitcoin\'s value. Cryptocurrency market caps follow a trend known as "halving", causing increases and decreases in value. Cryptocurrency investments are volatile and can be influenced by factors such as inflation warnings. Only a few leading cryptocurrencies have maintained their position in the market over the long term. It is essential to be aware of the risks associated with investing in cryptocurrency, as they have been compared to Ponzi schemes and bubbles. Conducting due diligence and staying informed about regulatory warnings and risks is crucial when investing